# 2d

In [29]:
from hyper_tuning_helper_functions import (
    model_mlp_2_classes,
    fit_then_evaluate_model
)
from load_datasets import create_generators

## Transference combined model method

In [30]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Flatten

image_size = 299

train_generator_299x299, val_generator_299x299 = create_generators(
    (image_size, image_size)
)

Xception_model_global_avarage_pooling_output = Sequential(
    [
        Xception(
            weights="imagenet",
            include_top=False,
            input_shape=(image_size, image_size, 3),
        ),
        GlobalAveragePooling2D(),
    ]
)

Xception_model_flatten_output = Sequential(
    [
        Xception(
            weights="imagenet",
            include_top=False,
            input_shape=(image_size, image_size, 3),
        ),
        Flatten(),
    ]
)

In [31]:
X_train_feature_maps = Xception_model_global_avarage_pooling_output.predict(train_generator_299x299)

2022-05-11 11:44:27.632890: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [33]:
test_predict_generator = Xception_model_global_avarage_pooling_output.predict(train_generator_299x299)

In [34]:
test_predict_generator.shape

(1600, 2048)

In [35]:
import numpy as np

def extract_X_y_from_generator(generator):
    steps = generator.n//generator.batch_size
    X, y = [] , []
    for _ in range(steps):
        X_batch, y_batch = generator.next()
        X.extend(X_batch)
        y.extend(y_batch)
    return np.array(X), np.array(y)

def extract_feature_maps(model, generator):
    X, y = extract_X_y_from_generator(generator)
    return model.predict(X), y

In [37]:
X_train_avarage_pooling, y_train_avarage_pooling = extract_feature_maps(Xception_model_global_avarage_pooling_output, train_generator_299x299)

X_val_avarage_pooling, y_val_avarage_pooling = extract_feature_maps(Xception_model_global_avarage_pooling_output, val_generator_299x299)



X_train_flatten, y_train_flatten = extract_feature_maps(Xception_model_flatten_output, train_generator_299x299)

X_val_flatten, y_val_flatten = extract_feature_maps(Xception_model_flatten_output, val_generator_299x299)


X_train_avarage_pooling.shape, X_train_flatten.shape, X_val_avarage_pooling.shape, X_val_flatten.shape

2022-05-11 11:47:10.197127: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-11 11:47:50.067441: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-11 11:48:22.310163: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


((1600, 2048), (1600, 204800), (384, 2048), (368, 204800))

In [38]:
dense_dropout_pairs = [(512,0)]
model_1 = model_mlp_2_classes(dense_dropout_pairs, X_train_avarage_pooling.shape[1])
model_1.summary()

Model: "transferacne_model_2_classes"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_8 (Dropout)         (None, 2048)              0         
                                                                 
 dense_4 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 output_layer (Dense)        (None, 1)                 513       
                                                                 
Total params: 1,049,601
Trainable params: 1,049,601
Non-trainable params: 0
_________________________________________________________________


In [39]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopper = EarlyStopping(
    monitor="val_acc", mode="max", patience=10, restore_best_weights=True
)

model_1.fit(
        X_train_avarage_pooling,
        y_train_avarage_pooling,
        epochs=100000,
        callbacks=[early_stopper],
        validation_data=(X_val_avarage_pooling, y_val_avarage_pooling),
        verbose=1,
    )

Epoch 1/100000
 9/50 [====>.........................] - ETA: 0s - loss: 0.4233 - acc: 0.7847 

2022-05-11 11:48:51.889874: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50/50 [==============================] - 1s 9ms/step - loss: 0.1745 - acc: 0.9244 - val_loss: 0.0456 - val_acc: 0.9844
Epoch 2/100000
11/50 [=====>........................] - ETA: 0s - loss: 0.0418 - acc: 0.9830

2022-05-11 11:48:52.332308: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50/50 [==============================] - 0s 6ms/step - loss: 0.0557 - acc: 0.9775 - val_loss: 0.0298 - val_acc: 0.9922
Epoch 3/100000
50/50 [==============================] - 0s 6ms/step - loss: 0.0399 - acc: 0.9844 - val_loss: 0.0311 - val_acc: 0.9896
Epoch 4/100000
50/50 [==============================] - 0s 7ms/step - loss: 0.0254 - acc: 0.9900 - val_loss: 0.0346 - val_acc: 0.9922
Epoch 5/100000
50/50 [==============================] - 0s 6ms/step - loss: 0.0229 - acc: 0.9912 - val_loss: 0.0334 - val_acc: 0.9870
Epoch 6/100000
50/50 [==============================] - 0s 7ms/step - loss: 0.0140 - acc: 0.9950 - val_loss: 0.0347 - val_acc: 0.9922
Epoch 7/100000
50/50 [==============================] - 0s 6ms/step - loss: 0.0146 - acc: 0.9931 - val_loss: 0.0418 - val_acc: 0.9922
Epoch 8/100000
50/50 [==============================] - 0s 6ms/step - loss: 0.0109 - acc: 0.9969 - val_loss: 0.0427 - val_acc: 0.9922
Epoch 9/100000
50/50 [==============================] - 0s 6ms/step - loss: 0

In [41]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopper = EarlyStopping(
    monitor="val_acc", mode="max", patience=10, restore_best_weights=True
)
model_1 = model_mlp_2_classes(dense_dropout_pairs, X_train_flatten.shape[1])
model_1.summary()
model_1.fit(
        X_train_flatten,
        y_train_flatten,
        epochs=100000,
        callbacks=[early_stopper],
        validation_data=(X_val_flatten, y_val_flatten),
        verbose=1,
    )

Model: "transferacne_model_2_classes"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_10 (Dropout)        (None, 204800)            0         
                                                                 
 dense_5 (Dense)             (None, 512)               104858112 
                                                                 
 dropout_11 (Dropout)        (None, 512)               0         
                                                                 
 output_layer (Dense)        (None, 1)                 513       
                                                                 
Total params: 104,858,625
Trainable params: 104,858,625
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100000


2022-05-11 11:55:40.635848: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50/50 [==============================] - ETA: 0s - loss: 7.4920 - acc: 0.9231

2022-05-11 11:55:46.808696: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50/50 [==============================] - 8s 149ms/step - loss: 7.4920 - acc: 0.9231 - val_loss: 1.1662 - val_acc: 0.9864
Epoch 2/100000
50/50 [==============================] - 7s 140ms/step - loss: 0.5971 - acc: 0.9775 - val_loss: 1.0455 - val_acc: 0.9918
Epoch 3/100000
50/50 [==============================] - 7s 133ms/step - loss: 0.2442 - acc: 0.9894 - val_loss: 1.0327 - val_acc: 0.9918
Epoch 4/100000
50/50 [==============================] - 7s 133ms/step - loss: 0.0479 - acc: 0.9956 - val_loss: 1.0540 - val_acc: 0.9891
Epoch 5/100000
50/50 [==============================] - 7s 135ms/step - loss: 0.0699 - acc: 0.9962 - val_loss: 1.0965 - val_acc: 0.9918
Epoch 6/100000
50/50 [==============================] - 7s 135ms/step - loss: 0.0402 - acc: 0.9975 - val_loss: 0.9949 - val_acc: 0.9918
Epoch 7/100000
50/50 [==============================] - 7s 135ms/step - loss: 0.0510 - acc: 0.9987 - val_loss: 0.9074 - val_acc: 0.9918
Epoch 8/100000
50/50 [==============================] - 7s 134m

In [ ]:
# TODO 4: testa olika mlp's
# TODO 5: testa använd random forest istället för mlp